Alter file state table schema to contain date columns.

In [0]:
def append_to_sys_path(folder: str, base_folder: str = "weather_bundle") -> str:
    """
    Append the specified folder to the system path if it is not already present.
    Allows importing packages from the folder.
    """
    import re
    import os
    import sys

    if not (match := re.search(rf"(.+[\\/]{base_folder})", os.getcwd())):
        raise FileNotFoundError(f"Could not find {base_folder} in current path: {os.getcwd()}")

    base_path = match.group(1)
    for root, dirs, _ in os.walk(base_path):
        if "src" in dirs:
            base_path = root
            break

    if (path := os.path.join(base_path, folder)) not in sys.path:
        sys.path.append(path)
    return path


append_to_sys_path("src/config")

In [0]:
dbutils.widgets.dropdown("dev_tests", "", ["", "True", "False"])  # noqa: F821

dev_tests = dbutils.widgets.get("dev_tests")  # noqa: F821

if not dev_tests:
    dev_tests = False

dev_tests = bool(dev_tests)

print(f"{dev_tests=}")
print(f"{type(dev_tests)=}")

In [0]:
from pyspark.sql.functions import split, element_at, col
 

In [0]:
from config import Config

if dev_tests:
    import os
    os.environ["ENV"] = "prd"
    os.environ["STAGE"] = "sandbox"

config = Config()
catalog = config["weather"]["catalog_name"]
bronze_schema = config["weather"]["bronze_schema_name"]
noaa: dict = config["weather"]["noaa"]
file_status_table_name = noaa["metrics"]["bronze_file_status_table_name"]

file_status_table = f"{catalog}.{bronze_schema}.{file_status_table_name}"
file_status_table_tmp = file_status_table + "_tmp"

print(f"{file_status_table=}")
print(f"{file_status_table_tmp=}")

In [0]:
df = spark.read.table(file_status_table)
df.printSchema()

In [0]:
spark.sql(f"DROP TABLE IF EXISTS {file_status_table_tmp}")

new_df = df \
    .withColumn("path_parts", split(col("file_write_target_path"), "/")) \
    .withColumn("year", element_at(col("path_parts"), -5).cast("int")) \
    .withColumn("month", element_at(col("path_parts"), -4).cast("int")) \
    .withColumn("day", element_at(col("path_parts"), -3).cast("int")) \
    .withColumn("run", element_at(col("path_parts"), -2).cast("int")) \
    .withColumn("dataset", element_at(col("path_parts"), -6)) \
    .drop("path_parts")
    
[print(c) for c in new_df.schema]

(new_df.write.format("delta")
    .mode("overwrite")
    .partitionBy("year", "month", "day")
    .saveAsTable(file_status_table_tmp))

spark.sql(f"DROP TABLE IF EXISTS {file_status_table}")
spark.sql(f"ALTER TABLE {file_status_table_tmp} RENAME TO {file_status_table}")

print("Table schema altered successfully")